In [ ]:
# -*- coding: utf-8 -*-
import os, time, re
from datetime import datetime
import pandas as pd
from tqdm import tqdm
from langchain.prompts import PromptTemplate
from openai import AzureOpenAI

# ───────────────────────────
# 0. Environment (already set externally)
# ───────────────────────────
os.environ['OPENAI_API_VERSION'] = '2024-02-15-preview'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://upappliance.openai.azure.com'
os.environ['AZURE_OPENAI_API_KEY'] = '4gWgDnxVjVaIRQhPJFn2seBhFGhsgfZhpedg6J0lmWDdsltOAlD8JQQJ99AKACYeBjFXJ3w3AAABACOGylYc'
os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'] = "gpt-4o-mini"  # 또는 "up_gpt4o"

# ───────────────────────────
# 1. LLM
# ───────────────────────────
client = AzureOpenAI(
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
    api_key=os.environ['AZURE_OPENAI_API_KEY'],
    api_version=os.environ['OPENAI_API_VERSION']
)
DEPLOYMENT = os.environ['AZURE_OPENAI_DEPLOYMENT_NAME']

# ───────────────────────────
# 2. Prompt
# ───────────────────────────
ICC_PROMPT = PromptTemplate.from_template("""
[System]
당신은 가전제품 고객 피드백을 분류하는 전문가입니다. 아래 제공된 정의와 예시, 분류 규칙을 반드시 준수하여, 각 메시지를 Issues(오류), Complaints(불만), Comments(문의) 중 하나로 정확하게 분류하세요.
ICC_label은 Issues, Complaints, Comments 중 하나로 통일하세요.

[Input]
afterchange: {afterchange}

[출력 형식]
오직 한 줄만 출력하세요. 추가 텍스트/코드블록 금지.
ICC_label: (Issues 또는 Comments 또는 Complaints)  # 영문 라벨만
""")

# ───────────────────────────
# 3. Utility
# ───────────────────────────
def to_safe_str(v):
    return "" if (pd.isna(v) or v is None) else str(v)

def extract_label(text: str):
    m = re.search(r"ICC_label\s*:\s*([A-Za-z]+)", text)
    return m.group(1).strip() if m else "Unknown"

def azure_chat_call(prompt: str, system_msg: str = "You are an expert classifier.", temperature: float = 0):
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": prompt},
    ]
    for _ in range(3):
        try:
            resp = client.chat.completions.create(
                model=DEPLOYMENT,
                messages=messages,
                temperature=temperature
            )
            content = resp.choices[0].message.content.strip()
            usage = getattr(resp, "usage", None)
            return content, usage
        except Exception as e:
            print(f"⚠️ Chat error {e}, retrying...")
            time.sleep(2)
    raise RuntimeError("Azure Chat call failed after 3 attempts")

def invoke_with_token(prompt_tmpl: PromptTemplate, inputs: dict, step_name: str):
    prompt = prompt_tmpl.format(**inputs)
    content, usage = azure_chat_call(prompt)
    token_log = {}
    if usage:
        token_log = {
            "step": step_name,
            "total_tokens": usage.total_tokens,
            "prompt_tokens": usage.prompt_tokens,
            "completion_tokens": usage.completion_tokens,
        }
        print(f"💰[{step_name}] total={usage.total_tokens}, prompt={usage.prompt_tokens}, completion={usage.completion_tokens}")
    return {
        "raw": content,
        "label": extract_label(content),
        "token_log": token_log
    }

# ───────────────────────────
# 4. Main
# ───────────────────────────
def main():
    df = pd.read_csv("./ICC_labeled.csv", encoding="utf-8-sig")
    df = df.iloc[:5]
    if "ICC_label" not in df.columns:
        df["ICC_label"] = ""

    token_logs = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            result = invoke_with_token(
                ICC_PROMPT,
                {"afterchange": to_safe_str(row.get("afterchange", ""))},
                "ICC Classification"
            )
            df.at[idx, "ICC_label"] = result["label"]
            if result["token_log"]:
                token_logs.append(result["token_log"])
        except Exception as e:
            print(f"[{idx}] Error: {e}")
            df.at[idx, "ICC_label"] = "Error"

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_csv = f"./ICC_labeled_ver10.csv"
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")

    print(f"✅ Done! Saved to: {out_csv}")

if __name__ == "__main__":
    main()